In [1]:
# Import dependencies
import pandas as pd
import os

In [2]:
# Folder and file paths
folder_path = "/Users/harwinder/Desktop/Tableau_Project/Resources/"
file_names = ['sep_2024.csv', 'oct_2024.csv', 'nov_2024.csv']
file_paths = [os.path.join(folder_path, file_name) for file_name in file_names]

In [3]:
# Merge data
data_frames = []
for file_path in file_paths:
    data = pd.read_csv(file_path, nrows=5000)  # Limit to 5000 rows
    data_frames.append(data)
merged_dataset = pd.concat(data_frames, ignore_index=True)

In [4]:
# Check data types before cleaning
print("Data Types Before Cleaning:\n")
print(merged_dataset.dtypes)


Data Types Before Cleaning:

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object


In [5]:
# Handle missing values
merged_dataset['start_station_name'] = merged_dataset['start_station_name'].fillna('Unknown')
merged_dataset['end_station_name'] = merged_dataset['end_station_name'].fillna('Unknown')
merged_dataset['start_station_id'] = merged_dataset['start_station_id'].fillna(-1)
merged_dataset['end_station_id'] = merged_dataset['end_station_id'].fillna(-1)

In [6]:
# Drop duplicates
merged_dataset = merged_dataset.drop_duplicates()

In [7]:
# Convert to datetime
merged_dataset['started_at'] = pd.to_datetime(merged_dataset['started_at'], errors='coerce')
merged_dataset['ended_at'] = pd.to_datetime(merged_dataset['ended_at'], errors='coerce')

In [8]:
# Ensure IDs are numeric
merged_dataset['start_station_id'] = pd.to_numeric(merged_dataset['start_station_id'], errors='coerce')
merged_dataset['end_station_id'] = pd.to_numeric(merged_dataset['end_station_id'], errors='coerce')

In [9]:
# Add calculated columns
merged_dataset['trip_duration_minutes'] = (merged_dataset['ended_at'] - merged_dataset['started_at']).dt.total_seconds() / 60
merged_dataset['day_of_week'] = merged_dataset['started_at'].dt.day_name()
merged_dataset['hour_of_day'] = merged_dataset['started_at'].dt.hour

In [10]:
# Filter invalid data
merged_dataset = merged_dataset[merged_dataset['trip_duration_minutes'] > 0]


In [11]:
# Convert string columns to lowercase for consistency
merged_dataset['rideable_type'] = merged_dataset['rideable_type'].str.lower()
merged_dataset['member_casual'] = merged_dataset['member_casual'].str.lower()

In [12]:
# Rename columns
merged_dataset = merged_dataset.rename(columns={
    'ride_id': 'Ride Id', 'rideable_type': 'Rideable Type', 'started_at': 'Started At',
    'ended_at': 'Ended At', 'start_station_name': 'Start Station Name',
    'start_station_id': 'Start Station Id', 'end_station_name': 'End Station Name',
    'end_station_id': 'End Station Id', 'start_lat': 'Start Lat', 'start_lng': 'Start Lng',
    'end_lat': 'End Lat', 'end_lng': 'End Lng', 'member_casual': 'Member Casual'
})

In [13]:
# Drop rows with missing lat/lng
merged_dataset = merged_dataset.dropna(subset=['End Lat', 'End Lng'])

In [14]:
# Ensure data types are correct
expected_dtypes = {
    'Ride Id': 'str',
    'Rideable Type': 'category',
    'Started At': 'datetime64[ns]',
    'Ended At': 'datetime64[ns]',
    'Start Station Name': 'str',
    'Start Station Id': 'int64',
    'End Station Name': 'str',
    'End Station Id': 'int64',
    'Start Lat': 'float64',
    'Start Lng': 'float64',
    'End Lat': 'float64',
    'End Lng': 'float64',
    'Member Casual': 'category',
    'trip_duration_minutes': 'float64',
    'day_of_week': 'str',
    'hour_of_day': 'int64'
}

for column, dtype in expected_dtypes.items():
    if column in merged_dataset.columns and merged_dataset[column].dtype != dtype:
        try:
            merged_dataset[column] = merged_dataset[column].astype(dtype)
        except Exception as e:
            print(f"Could not convert {column} to {dtype}: {e}")

Could not convert Start Station Id to int64: Cannot convert non-finite values (NA or inf) to integer
Could not convert End Station Id to int64: Cannot convert non-finite values (NA or inf) to integer


In [16]:
# Replace NaN or invalid values with -1 before conversion
merged_dataset['Start Station Id'] = merged_dataset['Start Station Id'].fillna(-1)
merged_dataset['End Station Id'] = merged_dataset['End Station Id'].fillna(-1)

In [20]:
#convert columns to integer
merged_dataset['Start Station Id'] = merged_dataset['Start Station Id'].astype('int64')
merged_dataset['End Station Id'] = merged_dataset['End Station Id'].astype('int64')

In [21]:
# Check data types after conversion
print("\nData Types After Conversion:\n")
print(merged_dataset.dtypes)


Data Types After Conversion:

Ride Id                          object
Rideable Type                  category
Started At               datetime64[ns]
Ended At                 datetime64[ns]
Start Station Name               object
Start Station Id                  int64
End Station Name                 object
End Station Id                    int64
Start Lat                       float64
Start Lng                       float64
End Lat                         float64
End Lng                         float64
Member Casual                  category
trip_duration_minutes           float64
day_of_week                      object
hour_of_day                       int64
dtype: object


In [24]:
# Save the cleaned dataset
cleaned_file_path = "/Users/harwinder/Desktop/Tableau_Project/final_datasets/recent_trends_citibike_dataset.csv"
merged_dataset.to_csv(cleaned_file_path, index=False)
print(f"Cleaned data saved to: {cleaned_file_path}")

Cleaned data saved to: /Users/harwinder/Desktop/Tableau_Project/final_datasets/recent_trends_citibike_dataset.csv
